# MSRGuard ChatBot Workbench (Notebook UI)

Dieses Notebook stellt ein **interaktives UI (ipywidgets)** bereit, um deinen PLC-Knowledge-Graph-ChatBot separat zu testen und weiterzuentwickeln.

Was du damit machen kannst:

- KG laden (TTL) und **Tool-Registry** initialisieren
- Einzelne Tools **auswählen und mit JSON-Args ausführen** (inkl. Output)
- **Neue Tools/Methoden** als Code einfügen und direkt **registrieren** (ohne Neustart)
- Optional: einen **Chat-Loop** nutzen (Planner → Tools → Antwort), wenn `OPENAI_API_KEY` gesetzt ist
- Optional: **Incident Session** (EvD2 Bootstrap) aus `excH_chatbot.py` wie im Desktop-UI, aber im Notebook

> Hinweis: Das UI ist bewusst „Werkbank“-artig gehalten: Du kannst es leicht erweitern.


In [ ]:
# Optional: ipywidgets installieren (falls nötig)
# In vielen Jupyter-Setups ist ipywidgets bereits vorhanden.
# Falls Import fehlschlägt: pip install ipywidgets

import sys, os, json, textwrap
from pathlib import Path

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output, Markdown
except Exception as e:
    raise RuntimeError(
        "ipywidgets ist nicht verfügbar. Bitte installiere es via: pip install ipywidgets und starte den Kernel neu."
    ) from e

import os
import re
from pathlib import Path

DEFAULT_OPENAI_KEY_FILE = r"C:\Users\Alexander Verkhov\Desktop\OpenAI API Key.txt"

def set_openai_key_from_file(path: str = DEFAULT_OPENAI_KEY_FILE) -> str:
    """
    Liest den OpenAI API Key aus einer Datei und setzt os.environ["OPENAI_API_KEY"].
    Unterstützt typische Formate wie:
      - sk-...
      - sk-proj-...
      - OPENAI_API_KEY=sk-...
    """
    p = Path(path).expanduser().resolve()
    if not p.exists():
        raise FileNotFoundError(f"Key-Datei nicht gefunden: {p}")

    raw = p.read_text(encoding="utf-8", errors="ignore").strip()

    # Key extrahieren (robust gegen "OPENAI_API_KEY=..." und Whitespace)
    m = re.search(r"(sk-(?:proj-)?[A-Za-z0-9_\-]{20,})", raw)
    if not m:
        raise ValueError(
            "Kein OpenAI API Key im File gefunden. Erwartet z.B. 'sk-...' oder 'OPENAI_API_KEY=sk-...'."
        )

    key = m.group(1).strip()
    os.environ["OPENAI_API_KEY"] = key
    return key

# Einmalig setzen (für den aktuellen Kernel)
key = set_openai_key_from_file(DEFAULT_OPENAI_KEY_FILE)
print("OPENAI_API_KEY gesetzt:", bool(os.environ.get("OPENAI_API_KEY")))
print("Key Preview:", key[:7] + "..." + key[-4:])


In [2]:
# Helper: Python-Root finden und sys.path setzen
# Ziel: Finde das msrguard Paket in einem Layout wie:
#   <parent>/MSRGuard_Anpassung/python/msrguard/chatbot_core.py
# oder
#   <parent>/python/msrguard/chatbot_core.py
#
# Wir gehen NUR in das Parent-Verzeichnis von cwd und durchsuchen dessen direkte Unterordner.

from pathlib import Path
import sys
import importlib.util

def ensure_python_root_on_sys_path() -> Path:
    here = Path.cwd().resolve()
    parent = here.parent  # NUR eine Ebene hoch

    # Kandidaten: parent selbst + alle direkten Unterordner
    candidates = [parent] + [p for p in parent.iterdir() if p.is_dir()]

    # 1) <cand>/python/msrguard/chatbot_core.py
    for c in candidates:
        probe = c / "python" / "msrguard" / "chatbot_core.py"
        if probe.exists():
            root = c / "python"
            if str(root) not in sys.path:
                sys.path.insert(0, str(root))
            return root

    # 2) <cand>/msrguard/chatbot_core.py (seltenes Layout)
    for c in candidates:
        probe = c / "msrguard" / "chatbot_core.py"
        if probe.exists():
            root = c
            if str(root) not in sys.path:
                sys.path.insert(0, str(root))
            return root

    # 3) Fallback: parent
    root = parent
    if str(root) not in sys.path:
        sys.path.insert(0, str(root))
    return root

def load_module_from_path(path: Path, module_name: str):
    """Importiert ein .py File als Modul (Fallback wenn Paket-Import nicht klappt)."""
    path = Path(path).expanduser().resolve()
    if not path.exists():
        raise FileNotFoundError(f"Modul nicht gefunden: {path}")
    spec = importlib.util.spec_from_file_location(module_name, str(path))
    if spec is None or spec.loader is None:
        raise RuntimeError(f"Kann spec nicht erstellen für: {path}")
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)  # type: ignore
    return mod

python_root = ensure_python_root_on_sys_path()
print("cwd:", Path.cwd().resolve())
print("python_root:", python_root)
print("sys.path[0]:", sys.path[0])
print("expect chatbot_core:", (python_root / "msrguard" / "chatbot_core.py").exists(), python_root / "msrguard" / "chatbot_core.py")


cwd: D:\MA_Python_Agent\Notebooks
python_root: D:\MA_Python_Agent\MSRGuard_Anpassung\python
sys.path[0]: D:\MA_Python_Agent\MSRGuard_Anpassung\python
expect chatbot_core: True D:\MA_Python_Agent\MSRGuard_Anpassung\python\msrguard\chatbot_core.py


In [3]:
# Imports: msrguard.* bevorzugen, sonst Fallback über python_root/msrguard/*.py

# --- chatbot_core ---
try:
    from msrguard.chatbot_core import (
        ChatBot, ToolRegistry, BaseAgentTool,
        KGStore, RoutineIndex,
        ListProgramsTool, EvD2DiagnosisTool, CalledPousTool, PouCallersTool, PouCodeTool,
        SearchVariablesTool, VariableTraceTool, GeneralSearchTool, GraphInvestigateTool,
        StringTripleSearchTool, ExceptionAnalysisTool, Text2SparqlTool,
        build_vector_index, SemanticSearchTool,
        get_llm_invoke, schema_card,
    )
    import msrguard.chatbot_core as chatbot_core
    print("Imported msrguard.chatbot_core from:", chatbot_core.__file__)
except Exception:
    chatbot_core = load_module_from_path(python_root / "msrguard" / "chatbot_core.py", "chatbot_core")
    print("Fallback import chatbot_core from:", python_root / "msrguard" / "chatbot_core.py")

    ChatBot = chatbot_core.ChatBot
    ToolRegistry = chatbot_core.ToolRegistry
    BaseAgentTool = chatbot_core.BaseAgentTool
    KGStore = chatbot_core.KGStore
    RoutineIndex = chatbot_core.RoutineIndex
    ListProgramsTool = chatbot_core.ListProgramsTool
    EvD2DiagnosisTool = chatbot_core.EvD2DiagnosisTool
    CalledPousTool = chatbot_core.CalledPousTool
    PouCallersTool = chatbot_core.PouCallersTool
    PouCodeTool = chatbot_core.PouCodeTool
    SearchVariablesTool = chatbot_core.SearchVariablesTool
    VariableTraceTool = chatbot_core.VariableTraceTool
    GeneralSearchTool = chatbot_core.GeneralSearchTool
    GraphInvestigateTool = chatbot_core.GraphInvestigateTool
    StringTripleSearchTool = chatbot_core.StringTripleSearchTool
    ExceptionAnalysisTool = chatbot_core.ExceptionAnalysisTool
    Text2SparqlTool = chatbot_core.Text2SparqlTool
    build_vector_index = chatbot_core.build_vector_index
    SemanticSearchTool = chatbot_core.SemanticSearchTool
    get_llm_invoke = chatbot_core.get_llm_invoke
    schema_card = chatbot_core.schema_card

# --- excH_chatbot ---
try:
    from msrguard.excH_chatbot import IncidentContext, ExcHChatBotSession, run_initial_analysis
    import msrguard.excH_chatbot as excH_chatbot
    print("Imported msrguard.excH_chatbot from:", excH_chatbot.__file__)
except Exception:
    excH_chatbot = load_module_from_path(python_root / "msrguard" / "excH_chatbot.py", "excH_chatbot")
    print("Fallback import excH_chatbot from:", python_root / "msrguard" / "excH_chatbot.py")
    IncidentContext = excH_chatbot.IncidentContext
    ExcHChatBotSession = excH_chatbot.ExcHChatBotSession
    run_initial_analysis = excH_chatbot.run_initial_analysis

# rdflib nur nötig, wenn du KG lädst
try:
    from rdflib import Graph
except Exception as e:
    raise RuntimeError("rdflib fehlt. Bitte installieren via: pip install rdflib") from e


Imported msrguard.chatbot_core from: D:\MA_Python_Agent\MSRGuard_Anpassung\python\msrguard\chatbot_core.py
Imported msrguard.excH_chatbot from: D:\MA_Python_Agent\MSRGuard_Anpassung\python\msrguard\excH_chatbot.py


In [ ]:
import os
from pathlib import Path
from typing import Optional

def try_set_openai_key_from_file(path_str: str) -> Optional[str]:
    # Wenn Key schon im Environment ist, nichts tun (optional: überschreiben, falls gewünscht)
    if os.environ.get("OPENAI_API_KEY"):
        return None
    
    if not path_str:
        return "OPENAI_API_KEY fehlt und openai_api_key_file Pfad ist leer."

    p = Path(path_str).expanduser().resolve()
    if not p.exists():
        return f"openai_api_key_file existiert nicht: {p}"

    try:
        key = p.read_text(encoding="utf-8").strip()
    except Exception as e:
        return f"Fehler beim Lesen der Key-Datei: {e}"

    if not key:
        return f"openai_api_key_file ist leer: {p}"

    os.environ["OPENAI_API_KEY"] = key
    print(f"API Key erfolgreich geladen aus: {p}")
    return None

## UI starten

1) Setze den KG-Pfad (TTL)  
2) Optional: setze `openai_api_key_file` oder `OPENAI_API_KEY`  
3) Klicke **Build**  
4) Tools testen, Chat testen, neue Tools hinzufügen



In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Any, Dict, List, Tuple, Optional

import json
import datetime
import os

import ipywidgets as widgets
from IPython.display import display, Markdown

# Import rdfLib nur wenn nötig (Fallback im Code vorhanden, aber hier für Typsicherheit)
try:
    from rdflib import Graph
except ImportError:
    pass

class MSRGuardWorkbench:
    def __init__(self):
        # Runtime objects
        self.graph: Optional[Graph] = None
        self.kg_store: Optional[Any] = None
        self.routine_index: Optional[Any] = None
        self.registry: Optional[Any] = None
        self.bot: Optional[Any] = None
        self.session: Optional[Any] = None
        self.schema_card_text: str = ""
        self.llm_invoke = None

        # --- UI Widgets ---
        
        # Standard-Pfad für KG (falls vorhanden)
        default_kg = Path.cwd() / "Test_cleaned.ttl"
        self.kg_path = widgets.Text(
            value=str(default_kg.resolve()) if default_kg.exists() else "",
            description="KG TTL",
            layout=widgets.Layout(width="95%")
        )
        
        self.routine_index_dir = widgets.Text(
            value="",
            description="Index Dir",
            placeholder="optional (Default: TTL Ordner)",
            layout=widgets.Layout(width="95%")
        )
        self.enable_rag = widgets.Checkbox(value=False, description="RAG (semantic_search) aktivieren")
        self.openai_model = widgets.Text(value="gpt-4o-mini", description="Model")
        self.openai_temp = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.1, description="Temp")
        
        # --- HIER IST DIE KORREKTUR ---
        # Der Pfad zur Key-Datei ist nun voreingestellt:
        self.openai_key_file = widgets.Text(
            value=r"C:\Users\Alexander Verkhov\Desktop\OpenAI API Key.txt", 
            description="Key-File", 
            layout=widgets.Layout(width="95%")
        )

        self.build_mode = widgets.Dropdown(
            options=[("Tools-only (ohne LLM)", "tools"), ("ChatBot (Planner+LLM)", "llm")],
            value="tools",
            description="Mode"
        )

        self.btn_build = widgets.Button(description="Build", button_style="success")
        self.btn_build.on_click(self._on_build_clicked)

        self.status = widgets.HTML("<b>Status:</b> idle")

        # --- Tools tab ---
        self.tool_dropdown = widgets.Dropdown(options=[], description="Tool")
        self.tool_doc = widgets.Textarea(value="", description="Doc", layout=widgets.Layout(width="95%", height="180px"))
        self.tool_args = widgets.Textarea(value="{}", description="Args(JSON)", layout=widgets.Layout(width="95%", height="120px"))
        self.btn_run_tool = widgets.Button(description="Run Tool", button_style="primary")
        self.btn_run_tool.on_click(self._on_run_tool_clicked)
        self.out_tool = widgets.Output(layout=widgets.Layout(border="1px solid #ddd", padding="8px"))

        self.tool_dropdown.observe(self._on_tool_selected, names="value")

        # --- Tool Builder tab ---
        self.newtool_name = widgets.Text(value="my_new_tool", description="name")
        self.newtool_desc = widgets.Text(value="Kurzbeschreibung...", description="description", layout=widgets.Layout(width="95%"))

        self.btn_skeleton = widgets.Button(description="Generate Skeleton", button_style="")
        self.btn_skeleton.on_click(self._on_generate_skeleton)

        self.btn_register = widgets.Button(description="Register Tool", button_style="warning")
        self.btn_register.on_click(self._on_register_tool_clicked)

        self.btn_save_tool = widgets.Button(description="Save to custom_tools.py", button_style="")
        self.btn_save_tool.on_click(self._on_save_tool_clicked)

        self.custom_code = widgets.Textarea(
            value="""# Schreibe hier deinen Tool-Code.
# Wichtig: definiere eine Funktion make_tool(ctx), die eine Tool-Instanz zurückgibt.
# ctx enthält: registry, kg_store, routine_index, graph, llm_invoke, schema_card_text, chatbot_core

""",
            description="code",
            layout=widgets.Layout(width="95%", height="320px")
        )
        self.out_builder = widgets.Output(layout=widgets.Layout(border="1px solid #ddd", padding="8px"))

        # --- Chat tab ---
        self.chat_debug = widgets.Checkbox(value=True, description="Debug (plan + tool_results anzeigen)")
        self.include_bootstrap = widgets.Checkbox(value=False, description="include_bootstrap (EvD2)")

        self.chat_input = widgets.Text(value="", description="User", layout=widgets.Layout(width="95%"), placeholder="Frage an den ChatBot...")
        self.btn_send = widgets.Button(description="Send", button_style="primary")
        self.btn_send.on_click(self._on_send_clicked)

        self.btn_clear_chat = widgets.Button(description="Clear", button_style="")
        self.btn_clear_chat.on_click(self._on_clear_chat)

        self.out_chat = widgets.Output(layout=widgets.Layout(border="1px solid #ddd", padding="8px", height="320px", overflow_y="auto"))

        # --- Incident tab ---
        # Optional: Event-Datei aus agent_results laden (z.B. evD2-..._event.json)
        self.event_file_path = widgets.Text(
            value="",
            description="Event File",
            placeholder=r"z.B. D:\MA_Python_Agent\MSRGuard_Anpassung\python\agent_results\..._event.json",
            layout=widgets.Layout(width="95%")
        )
        self.btn_load_event_file = widgets.Button(description="Load Event File", button_style="")
        self.btn_load_event_file.on_click(self._on_load_event_file)

        self.patch_event_kg = widgets.Checkbox(value=True, description="payload.kg_ttl_path mit KG TTL überschreiben")

        self.event_json = widgets.Textarea(
            value="""{
  "payload": {
    "triggerEvent": "evD2",
    "triggerD2": true,
    "lastExecutedSkill": "TestSkill3",
    "kg_ttl_path": ""
  }
}
""",
            description="Event JSON",
            layout=widgets.Layout(width="95%", height="240px")
        )
        self.btn_load_incident = widgets.Button(description="Load Incident Session", button_style="info")
        self.btn_load_incident.on_click(self._on_load_incident)

        self.btn_initial = widgets.Button(description="Run Initial Analysis", button_style="primary")
        self.btn_initial.on_click(self._on_run_initial_analysis)

        self.out_incident = widgets.Output(layout=widgets.Layout(border="1px solid #ddd", padding="8px"))

        # Tabs
        self.tab_config = widgets.VBox([
            widgets.HTML("<h3>Config</h3>"),
            widgets.HTML("<p><b>KG TTL</b> ist dein Knowledge Graph. <b>Index Dir</b> ist der Ordner für den RoutineIndex Cache (*_routine_index.json). Leer bedeutet: im TTL Ordner.</p>"),
            self.kg_path,
            self.routine_index_dir,
            widgets.HBox([self.build_mode, self.enable_rag]),
            widgets.HBox([self.openai_model, self.openai_temp]),
            self.openai_key_file,
            widgets.HBox([self.btn_build, self.status]),
        ])

        self.tab_tools = widgets.VBox([
            widgets.HTML("<h3>Tools</h3>"),
            self.tool_dropdown,
            self.tool_doc,
            self.tool_args,
            widgets.HBox([self.btn_run_tool]),
            self.out_tool,
        ])

        self.tab_builder = widgets.VBox([
            widgets.HTML("<h3>Tool Builder</h3>"), 
            widgets.HBox([self.newtool_name, self.btn_skeleton]),
            self.newtool_desc,
            self.custom_code,
            widgets.HBox([self.btn_register, self.btn_save_tool]),
            self.out_builder,
        ])

        self.tab_chat = widgets.VBox([
            widgets.HTML("<h3>Chat</h3>"),
            widgets.HBox([self.chat_debug, self.include_bootstrap, self.btn_clear_chat]),
            self.chat_input,
            self.btn_send,
            self.out_chat,
        ])

        self.tab_incident = widgets.VBox([
            widgets.HTML("<h3>Incident Session (optional)</h3>"), 
            widgets.HTML("<p>Erzeugt eine ExcHChatBotSession wie im Desktop-UI (run_initial_analysis nutzt EvD2 Bootstrap).</p>"),
            self.event_file_path,
            widgets.HBox([self.btn_load_event_file, self.patch_event_kg]),
            self.event_json,
            widgets.HBox([self.btn_load_incident, self.btn_initial]),
            self.out_incident
        ])

        self.tabs = widgets.Tab(children=[self.tab_config, self.tab_tools, self.tab_builder, self.tab_chat, self.tab_incident])
        for i, title in enumerate(["Config", "Tools", "Tool Builder", "Chat", "Incident"]):
            self.tabs.set_title(i, title)

    # ---------- Core build ----------
    def _load_graph(self, kg_ttl_path: str) -> Graph:
        ttl = Path(kg_ttl_path).expanduser().resolve()
        if not ttl.exists():
            raise FileNotFoundError(f"KG TTL nicht gefunden: {ttl}")
        g = Graph()
        g.parse(str(ttl), format="turtle")
        return g

    def _build_tools_only(self):
        kg_ttl_path = self.kg_path.value.strip()
        self.graph = self._load_graph(kg_ttl_path)

        # wichtig: module-global setzen, da chatbot_core.sparql_select_raw auf 'g' zugreift
        try:
            chatbot_core.g = self.graph
        except Exception:
            pass

        self.kg_store = KGStore(self.graph)
        self.schema_card_text = schema_card(self.graph, top_n=15)

        # Routine index wie im build_bot
        idx_dir = Path(self.routine_index_dir.value.strip()).expanduser().resolve() if self.routine_index_dir.value.strip() else Path(kg_ttl_path).expanduser().resolve().parent
        idx_dir.mkdir(parents=True, exist_ok=True)
        idx_path = idx_dir / (Path(kg_ttl_path).stem + "_routine_index.json")

        if idx_path.exists() and idx_path.stat().st_size > 0:
            self.routine_index = RoutineIndex.load(str(idx_path))
        else:
            self.routine_index = RoutineIndex.build_from_kg(self.kg_store)
            self.routine_index.save(str(idx_path))

        self.registry = ToolRegistry()
        # Deterministische Tools
        self.registry.register(ListProgramsTool())
        self.registry.register(EvD2DiagnosisTool())
        self.registry.register(CalledPousTool())
        self.registry.register(PouCallersTool())
        self.registry.register(PouCodeTool())
        self.registry.register(SearchVariablesTool())
        self.registry.register(VariableTraceTool())
        self.registry.register(GeneralSearchTool())
        self.registry.register(GraphInvestigateTool())
        self.registry.register(StringTripleSearchTool(self.kg_store))
        self.registry.register(ExceptionAnalysisTool(self.kg_store, self.routine_index))

        self.bot = None
        self.llm_invoke = None
        self.session = None

    def _build_llm_bot(self):
        # Key setzen (falls Key-File angegeben)
        err = try_set_openai_key_from_file(self.openai_key_file.value.strip())
        if err:
            # Nur Hinweis, kein Abbruch: User kann OPENAI_API_KEY auch anders setzen
            print("Key Hinweis:", err)

        kg_ttl_path = self.kg_path.value.strip()
        self.graph = self._load_graph(kg_ttl_path)

        try:
            chatbot_core.g = self.graph
        except Exception:
            pass

        self.kg_store = KGStore(self.graph)
        self.schema_card_text = schema_card(self.graph, top_n=15)

        idx_dir = Path(self.routine_index_dir.value.strip()).expanduser().resolve() if self.routine_index_dir.value.strip() else Path(kg_ttl_path).expanduser().resolve().parent
        idx_dir.mkdir(parents=True, exist_ok=True)
        idx_path = idx_dir / (Path(kg_ttl_path).stem + "_routine_index.json")

        if idx_path.exists() and idx_path.stat().st_size > 0:
            self.routine_index = RoutineIndex.load(str(idx_path))
        else:
            self.routine_index = RoutineIndex.build_from_kg(self.kg_store)
            self.routine_index.save(str(idx_path))

        # Hier würde der Error fliegen, wenn der Key nicht geladen wurde
        self.llm_invoke = get_llm_invoke(model=self.openai_model.value.strip() or "gpt-4o-mini", temperature=float(self.openai_temp.value))

        self.registry = ToolRegistry()
        self.registry.register(ListProgramsTool())
        self.registry.register(EvD2DiagnosisTool())
        self.registry.register(CalledPousTool())
        self.registry.register(PouCallersTool())
        self.registry.register(PouCodeTool())
        self.registry.register(SearchVariablesTool())
        self.registry.register(VariableTraceTool())
        self.registry.register(GeneralSearchTool())
        self.registry.register(GraphInvestigateTool())
        self.registry.register(StringTripleSearchTool(self.kg_store))
        self.registry.register(ExceptionAnalysisTool(self.kg_store, self.routine_index))
        self.registry.register(Text2SparqlTool(self.llm_invoke, self.schema_card_text))

        if bool(self.enable_rag.value):
            vs = build_vector_index(self.kg_store, self.registry)
            if vs is not None:
                self.registry.register(SemanticSearchTool(vs))

        self.bot = ChatBot(self.registry, self.llm_invoke)
        self.session = None

    def _refresh_tool_list(self):
        if not self.registry:
            self.tool_dropdown.options = []
            self.tool_doc.value = ""
            return

        names = sorted(list(getattr(self.registry, "_tools", {}).keys()))
        self.tool_dropdown.options = names
        if names:
            self.tool_dropdown.value = names[0]
            self._update_tool_doc(names[0])

    def _update_tool_doc(self, tool_name: str):
        if not self.registry:
            self.tool_doc.value = ""
            return
        tool = getattr(self.registry, "_tools", {}).get(tool_name)
        if tool is None:
            self.tool_doc.value = ""
            return
        try:
            self.tool_doc.value = tool.get_documentation()
        except Exception:
            self.tool_doc.value = f"{tool_name}\n(no documentation)\n{tool}"

    # ---------- UI callbacks ----------
    def _set_status(self, text: str):
        self.status.value = f"<b>Status:</b> {text}"

    def _on_build_clicked(self, _btn):
        with self.out_tool:
            pass
        try:
            self._set_status("building...")
            mode = self.build_mode.value
            if mode == "tools":
                self._build_tools_only()
                self._set_status("ready (tools-only)")
            else:
                self._build_llm_bot()
                self._set_status("ready (llm chatbot)")
            self._refresh_tool_list()
        except Exception as e:
            self._set_status(f"ERROR: {e}")
            raise

    def _on_tool_selected(self, change):
        val = change.get("new")
        if val:
            self._update_tool_doc(val)

    def _on_run_tool_clicked(self, _btn):
        self.out_tool.clear_output()
        with self.out_tool:
            if not self.registry:
                print("Registry ist nicht gebaut. Erst in 'Config' auf Build klicken.")
                return
            tool_name = self.tool_dropdown.value
            try:
                args = json.loads(self.tool_args.value.strip() or "{}")
                if not isinstance(args, dict):
                    args = {}
            except Exception as e:
                print("Args JSON parse error:", e)
                return

            res = self.registry.execute(tool_name, args)
            print(json.dumps(res, ensure_ascii=False, indent=2) if not isinstance(res, str) else res)

    def _on_generate_skeleton(self, _btn):
        tool_name = (self.newtool_name.value or "my_new_tool").strip()
        desc = (self.newtool_desc.value or "Kurzbeschreibung...").strip()

        skel = f'''# --- Custom Tool Skeleton ---
# ctx enthält: registry, kg_store, routine_index, graph, llm_invoke, schema_card_text, chatbot_core
# Du MUSST make_tool(ctx) definieren, die eine Tool-Instanz zurückgibt.

class {tool_name.title().replace("_", "")}Tool(chatbot_core.BaseAgentTool):
    name = "{tool_name}"
    description = "{desc}"
    usage_guide = "Wann soll der Planner dieses Tool nutzen?"

    def __init__(self, kg_store=None, routine_index=None, registry=None, llm_invoke=None):
        self.kg_store = kg_store
        self.routine_index = routine_index
        self.registry = registry
        self.llm_invoke = llm_invoke

    def run(self, **kwargs):
        # TODO: Implementiere hier deine Logik.
        # Du kannst z.B. über self.registry.execute("pou_code", {{...}}) andere Tools nutzen.
        return {{
            "ok": True,
            "note": "Implement me",
            "kwargs": kwargs
        }}

def make_tool(ctx):
    return {tool_name.title().replace("_", "")}Tool(
        kg_store=ctx.get("kg_store"),
        routine_index=ctx.get("routine_index"),
        registry=ctx.get("registry"),
        llm_invoke=ctx.get("llm_invoke"),
    )
'''
        self.custom_code.value = skel

    def _on_register_tool_clicked(self, _btn):
        self.out_builder.clear_output()
        with self.out_builder:
            if not self.registry:
                print("Registry ist nicht gebaut. Erst in 'Config' auf Build klicken.")
                return

            code = self.custom_code.value
            ctx = {
                "registry": self.registry,
                "kg_store": self.kg_store,
                "routine_index": self.routine_index,
                "graph": self.graph,
                "llm_invoke": self.llm_invoke,
                "schema_card_text": self.schema_card_text,
                "chatbot_core": chatbot_core,
            }

            ns: Dict[str, Any] = {}
            try:
                exec(code, ns, ns)
            except Exception as e:
                print("Code exec error:", e)
                return

            if "make_tool" not in ns:
                print("Fehlt: make_tool(ctx). Bitte im Code definieren.")
                return

            try:
                tool_obj = ns["make_tool"](ctx)
            except Exception as e:
                print("make_tool(ctx) error:", e)
                return

            if not hasattr(tool_obj, "name") or not hasattr(tool_obj, "run"):
                print("make_tool(ctx) muss ein Tool-Objekt (BaseAgentTool) zurückgeben.")
                return

            self.registry.register(tool_obj)
            print(f"Registered tool: {tool_obj.name}")
            self._refresh_tool_list()

    def _on_save_tool_clicked(self, _btn):
        self.out_builder.clear_output()
        with self.out_builder:
            code = self.custom_code.value.strip()
            if not code:
                print("Kein Code zum Speichern.")
                return
            outp = Path("custom_tools.py").resolve()
            stamp = f"\n\n# ---- saved {datetime.datetime.utcnow().isoformat()}Z ----\n"
            outp.write_text((outp.read_text(encoding="utf-8") if outp.exists() else "") + stamp + code + "\n", encoding="utf-8")
            print(f"Saved to: {outp}")

    def _append_chat(self, role: str, text: str):
        with self.out_chat:
            display(Markdown(f"**{role}:**\n\n{text}\n"))

    def _on_clear_chat(self, _btn):
        self.out_chat.clear_output()

    def _on_send_clicked(self, _btn):
        msg = self.chat_input.value.strip()
        if not msg:
            return
        self.chat_input.value = ""

        self._append_chat("User", msg)

        if self.session is not None:
            # Incident session
            try:
                res = self.session.ask(msg, debug=bool(self.chat_debug.value), include_bootstrap=bool(self.include_bootstrap.value))
                answer = res.get("answer") if isinstance(res, dict) else str(res)
                self._append_chat("Assistant", answer or "")
                if bool(self.chat_debug.value) and isinstance(res, dict):
                    plan = res.get("plan")
                    tool_results = res.get("tool_results")
                    self._append_chat("Debug/Plan", f"```json\n{json.dumps(plan, ensure_ascii=False, indent=2)}\n```" if plan is not None else "(none)")
                    self._append_chat("Debug/Tools", f"```json\n{json.dumps(tool_results, ensure_ascii=False, indent=2)[:12000]}\n```" if tool_results is not None else "(none)")
            except Exception as e:
                self._append_chat("System", f"ChatBot Fehler: {e}")
            return

        # Standard bot chat
        if not self.bot:
            self._append_chat("System", "ChatBot ist nicht initialisiert (LLM-Mode). Baue den Bot in 'Config' im LLM-Mode.")
            return

        try:
            res = self.bot.chat(msg, debug=bool(self.chat_debug.value))
            answer = res.get("answer") if isinstance(res, dict) else str(res)
            self._append_chat("Assistant", answer or "")
            if bool(self.chat_debug.value) and isinstance(res, dict):
                plan = res.get("plan")
                tool_results = res.get("tool_results")
                self._append_chat("Debug/Plan", f"```json\n{json.dumps(plan, ensure_ascii=False, indent=2)}\n```" if plan is not None else "(none)")
                self._append_chat("Debug/Tools", f"```json\n{json.dumps(tool_results, ensure_ascii=False, indent=2)[:12000]}\n```" if tool_results is not None else "(none)")
        except Exception as e:
            self._append_chat("System", f"ChatBot Fehler: {e}")

    def _on_load_event_file(self, _btn):
        """Lädt ein *_event.json aus einer Datei in das Event JSON Textfeld."""
        self.out_incident.clear_output()
        with self.out_incident:
            p = Path(self.event_file_path.value.strip()).expanduser().resolve()
            if not p.exists():
                print("Event File nicht gefunden:", p)
                return
            try:
                ev = json.loads(p.read_text(encoding="utf-8"))
            except Exception as e:
                print("Event File JSON parse error:", e)
                return

            # Optional: kg_ttl_path im Event setzen
            if bool(self.patch_event_kg.value):
                kgp = self.kg_path.value.strip()
                if kgp:
                    ev.setdefault("payload", {})
                    if isinstance(ev["payload"], dict):
                        ev["payload"]["kg_ttl_path"] = kgp
                        print("payload.kg_ttl_path gesetzt auf:", kgp)

            self.event_json.value = json.dumps(ev, ensure_ascii=False, indent=2)
            print("Event File geladen:", p)

    def _on_load_incident(self, _btn):
        self.out_incident.clear_output()
        with self.out_incident:
            if not self.bot and not self.registry:
                print("Bitte zuerst in Config den Bot/Registry bauen.")
                return

            try:
                ev = json.loads(self.event_json.value)
            except Exception as e:
                print("Event JSON parse error:", e)
                return

            # KG Pfad aus event payload kann Config überschreiben
            payload = ev.get("payload") if isinstance(ev.get("payload"), dict) else {}
            kgp = payload.get("kg_ttl_path") or ""
            if kgp and kgp.strip():
                self.kg_path.value = str(kgp)
                print("KG TTL aus Event übernommen:", kgp)
                # rebuild bot/registry (same mode)
                self._on_build_clicked(None)

            # Lazy import (robust): IncidentContext + Session immer verfügbar machen
            try:
                from msrguard.excH_chatbot import IncidentContext as _IncidentContext, ExcHChatBotSession as _ExcHChatBotSession
            except Exception:
                _excH = load_module_from_path(python_root / "msrguard" / "excH_chatbot.py", "excH_chatbot")
                _IncidentContext = _excH.IncidentContext
                _ExcHChatBotSession = _excH.ExcHChatBotSession

            ctx = _IncidentContext.from_event(ev) if hasattr(_IncidentContext, "from_event") else _IncidentContext.from_input(ev)
            # Session braucht bot; falls nur tools-mode, versuche trotzdem eine Minimal-ChatBot-Instanz ohne LLM
            if self.bot is None:
                # tools-only: pseudo-bot mit registry, aber ohne llm
                dummy_llm = lambda _sys, _user: ""
                self.bot = ChatBot(self.registry, dummy_llm)  # type: ignore

            self.session = _ExcHChatBotSession(bot=self.bot, ctx=ctx)
            print("Incident Session geladen.")
            print("ctx:", ctx)

    def _on_run_initial_analysis(self, _btn):
        self.out_incident.clear_output()
        with self.out_incident:
            if self.session is None:
                print("Keine Incident Session. Erst 'Load Incident Session' klicken.")
                return
            try:
                try:
                    from msrguard.excH_chatbot import run_initial_analysis as _run_initial_analysis
                except Exception:
                    _excH = load_module_from_path(python_root / "msrguard" / "excH_chatbot.py", "excH_chatbot")
                    _run_initial_analysis = _excH.run_initial_analysis

                res = _run_initial_analysis(self.session, debug=True)
                print("Initial analysis done.")
                print("Answer:\n", res.get("answer") if isinstance(res, dict) else res)
                if isinstance(res, dict):
                    print("\nPlan:\n", json.dumps(res.get("plan"), ensure_ascii=False, indent=2))
                    print("\nTool results:\n", json.dumps(res.get("tool_results"), ensure_ascii=False, indent=2)[:12000])
            except Exception as e:
                print("run_initial_analysis error:", e)

    def display(self):
        display(self.tabs)

# UI starten:
ui = MSRGuardWorkbench()
ui.display()

FileNotFoundError: KG TTL nicht gefunden: D:\MA_Python_Agent\Notebooks\"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl"

FileNotFoundError: KG TTL nicht gefunden: D:\KGs\TestEvents.ttl

FileNotFoundError: KG TTL nicht gefunden: D:\MA_Python_Agent\Notebooks\"D:\MA_Python_Agent\MSRGuard_Anpassung\KGs\TestEvents.ttl"

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable